In [1]:
# 기존 하이퍼 파라미터 최적화
# 피처 엔지니어링을 포함해서 
# 여기에서 모델은 LightGBM -> XGBoost 로 바꾸었을뿐 성능 향상 측정 하는것이 목표 

In [8]:
import os.path
from os import path
import pandas as pd

# 캐글 ↓제출경로
data_path = './input/'

train = pd.read_csv(data_path + 'train.csv', index_col='id')
test  = pd.read_csv(data_path + 'test.csv' , index_col='id')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col='id')

all_data = pd.concat([train, test], ignore_index=True)
all_data = all_data.drop('target', axis=1) # 타깃값 제거 
all_features = all_data.columns

# 명목형 피처는 보통 웟-핫 인코딩 적용하는게 정석인듯 하다 
# 명목형 피처 추출
from sklearn.preprocessing import OneHotEncoder
cat_features = [feature for feature in all_features if 'cat' in feature]

# 원 핫 인코딩
onehot_encoder = OneHotEncoder()
encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features])

# 파생 피처 추가 
# 데이터 하나당 결측값 개수를 파생 피처로 추가

all_data['num_missing'] = (all_data==-1).sum(axis=1)

# 명목형 피처, calc 분류의 피처를 제외한 피처
remaining_features = [
    feature for feature in all_features
    if ('cat' not in feature and 'calc' not in feature)
]

# num_missing 을 remaining_features 에 추가
remaining_features.append('num_missing')

# 갑자기 뜬금없는 피처 엔지니어링 

ind_features = [feature for feature in all_features if 'ind' in feature]

is_first_feature = True
for ind_featrue in ind_features:
    if is_first_feature:
        all_data['mix_ind'] = all_data[ind_featrue].astype(str) + '_'
        is_first_feature = False
    else:
        all_data['mix_ind'] += all_data[ind_featrue].astype(str) + '_'
        
all_data['mix_ind']

cat_count_features = []
for feature in cat_features + ['mix_ind']:
    val_counts_dict = all_data[feature].value_counts().to_dict()
    all_data[f'{feature}_count'] = all_data[feature].apply(lambda x: val_counts_dict[x])
    
    cat_count_features.append(f'{feature}_count')
    
# 필요없다고 판단한 피처는 제거한 다음 모든 데이터 합치기 
from scipy import sparse 

# 필요 없는 피처들
drop_features = [
    'ps_ind_14', 'ps_ind_10_bin', 'ps_ind_11_bin',
    'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_car_14'
]

# remaining_features, cat_count_features 에서 drop_features 를 제거한 데이터 
all_data_remaining = all_data[remaining_features + cat_count_features].drop(drop_features, axis=1)

# 데이터 합치기
all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data_remaining),
                               encoded_cat_matrix],
                               format='csr'
                               )

# 데이터 나누기
num_train = len(train)

X = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train:]

y = train['target'].values 

In [9]:
import numpy as np

def eval_gini(y_true, y_pred):
    # 실젯값과 예측값의 크기가 서로 같은지 확인 (값이 다르면 오류 발생 )
    
    assert y_true.shape == y_pred.shape
    
    n_samples = y_true.shape[0] # 데이터 개수
    L_mid = np.linspace( 1/ n_samples, 1, n_samples) # 1/n_samples~ 1 까지 n_samples-1 개의 구간으로 나누겠다. 
    
    # 1) 예측값에 대한 지니계수
    pred_order = y_true[y_pred.argsort()]
    L_pred = np.cumsum(pred_order) / np.sum(pred_order) # 로렌츠 곡선
    G_pred = np.sum(L_mid - L_pred)
    
    # 2) 예측 완벽할때 지니계수
    true_order = y_true[y_true.argsort()]
    L_true = np.cumsum(true_order) / np.sum(true_order)
    G_true = np.sum(L_mid - L_true)
    
    # 정규화된 지니계수
    return G_pred / G_true

# XGBoost 용 gini() 함수
def gini(preds, dtrain):
    labels = dtrain.get_label()
    return 'gini', eval_gini(labels, preds)

In [10]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

# 8:2 비율로 훈련데이터, 검증 데이터 분리 (베이지안 최적화 수행용)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)

# 베이지안 최적화용 데이터셋
bayes_dtrain = xgb.DMatrix(X_train, y_train)
bayes_dvalid = xgb.DMatrix(X_valid, y_valid)

In [11]:
# 하이퍼 파라미터 범위 설정
# 베이지안 최적화를 위한 하이퍼 파라미터 범위

param_bound = {
    'max_depth' : (4,8),
    'subsample' : (0.6, 0.9),
    'colsample_bytree': (0.7, 1.0),
    'min_child_weight': (5, 7),
    'gamma': (8,11),
    'reg_alpha' : (7,9),
    'reg_lambda': (1.1, 1.5),
    'scale_pos_weight': (1.4, 1.6)
}

# 값이 고정된 하이퍼 파라미터
fixed_param ={
    'objective': 'binary:logistic',
    'learning_rate': 0.02,
    'random_state': 1991
}

In [13]:
# 기존 평과함수 
# LightGBM 용 eval_function() 과 유사하나 다른점은 다음과 같다.

# 하이퍼 파라미터명
# train() 메서드 내 검증 데이터 전달 방식
# train() 메서드 내 maximize 파라미터 
# predcit() 메서드에 DMatrix 타입을 전달하는 점 

def eval_function(max_depth, subsample, colsample_bytree, min_child_weight, 
                  reg_alpha, gamma, reg_lambda, scale_pos_weight):
    '''최적화하려는 평가지표(지니계수) 계산 함수'''
    # 베이지안 최적화를 수행할 하이퍼파라미터 
    params = {
        'max_depth': int(round(max_depth)),
        'subsample': subsample,
        'colsample_bytree': colsample_bytree,
        'min_child_weight': min_child_weight,
        'gamma': gamma,
        'reg_alpha': reg_alpha,
        'reg_lambda': reg_lambda,
        'scale_pos_weight': scale_pos_weight
    }
    # 값이 고정된 하이퍼 파라미터도 추가
    params.update(fixed_param)
    
    print('하이퍼 파라미터 : ', params)
    
    # XGBoost 모델 훈련
    xgb_model = xgb.train(params=params,
                          dtrain=bayes_dtrain,
                          num_boost_round=2000,
                          evals=[(bayes_dvalid,'bayes_dvalid')],
                          maximize=True,
                          feval=gini,
                          early_stopping_rounds=200,
                          verbose_eval=False
                          )
    
    best_iter = xgb_model.best_iteration # 최적 반복 횟수
    # 검증 데이터로 예측 수행
    preds = xgb_model.predict(bayes_dvalid, iteration_range=(0, best_iter))
    
    # 지니계수 계산
    gini_score = eval_gini(y_valid, preds)
    print(f'지니계수 : {gini_score}\n')
    
    return gini_score
    

In [14]:
# 최적화 수행
from bayes_opt import BayesianOptimization

# 베이지안 최적화 객체 생성
optimizer = BayesianOptimization(f=eval_function,
                                 pbounds=param_bound,
                                 random_state=0)

# 베이지안 최적화 수행
optimizer.maximize(init_points=3 ,n_iter=6)

|   iter    |  target   | colsam... |   gamma   | max_depth | min_ch... | reg_alpha | reg_la... | scale_... | subsample |
-------------------------------------------------------------------------------------------------------------------------
하이퍼 파라미터 :  {'max_depth': 6, 'subsample': 0.867531900234624, 'colsample_bytree': 0.8646440511781974, 'min_child_weight': 6.0897663659937935, 'gamma': 10.14556809911726, 'reg_alpha': 7.84730959867781, 'reg_lambda': 1.3583576452266626, 'scale_pos_weight': 1.4875174422525386, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'random_state': 1991}


c:\ProgramData\Anaconda3\lib\site-packages\xgboost\training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


지니계수 : 0.2852804659784522

| 1         | 0.2853    | 0.8646    | 10.15     | 6.411     | 6.09      | 7.847     | 1.358     | 1.488     | 0.8675    |
하이퍼 파라미터 :  {'max_depth': 7, 'subsample': 0.6261387899104622, 'colsample_bytree': 0.9890988281503088, 'min_child_weight': 6.0577898395058085, 'gamma': 9.150324556477333, 'reg_alpha': 8.136089122187865, 'reg_lambda': 1.4702386553170643, 'scale_pos_weight': 1.4142072116395774, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'random_state': 1991}
지니계수 : 0.2848088779310762

| 2         | 0.2848    | 0.9891    | 9.15      | 7.167     | 6.058     | 8.136     | 1.47      | 1.414     | 0.6261    |
하이퍼 파라미터 :  {'max_depth': 7, 'subsample': 0.8341587528859367, 'colsample_bytree': 0.7060655192320977, 'min_child_weight': 6.7400242964936385, 'gamma': 10.497859536643814, 'reg_alpha': 8.957236684465528, 'reg_lambda': 1.4196634256866894, 'scale_pos_weight': 1.4922958724505864, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'random_state': 1

c:\ProgramData\Anaconda3\lib\site-packages\xgboost\training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


지니계수 : 0.28372532851421195

| 4         | 0.2837    | 0.8843    | 10.45     | 6.838     | 6.494     | 8.552     | 1.381     | 1.423     | 0.7002    |
하이퍼 파라미터 :  {'max_depth': 7, 'subsample': 0.8535233675350644, 'colsample_bytree': 0.92975858050776, 'min_child_weight': 6.249564429359247, 'gamma': 9.95563546750357, 'reg_alpha': 8.411512219837842, 'reg_lambda': 1.424460008293778, 'scale_pos_weight': 1.5416807226581535, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'random_state': 1991}


...

지니계수 : 0.28467572646560496

| 7         | 0.2847    | 0.8818    | 9.013     | 6.927     | 6.992     | 7.641     | 1.356     | 1.449     | 0.6931    |
하이퍼 파라미터 :  {'max_depth': 5, 'subsample': 0.6261564417044092, 'colsample_bytree': 0.8763145220620449, 'min_child_weight': 5.135323353557588, 'gamma': 8.39495450163982, 'reg_alpha': 8.950443047087845, 'reg_lambda': 1.4235649099168255, 'scale_pos_weight': 1.5217625173811569, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'random_state': 1991}


c:\ProgramData\Anaconda3\lib\site-packages\xgboost\training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


지니계수 : 0.28457181017567373

| 8         | 0.2846    | 0.8763    | 8.395     | 4.561     | 5.135     | 8.95      | 1.424     | 1.522     | 0.6262    |
하이퍼 파라미터 :  {'max_depth': 6, 'subsample': 0.857971740304964, 'colsample_bytree': 0.9583821245229369, 'min_child_weight': 6.158305055403563, 'gamma': 9.305332775334449, 'reg_alpha': 8.200928434091152, 'reg_lambda': 1.2571039588093065, 'scale_pos_weight': 1.4700266933495618, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'random_state': 1991}


c:\ProgramData\Anaconda3\lib\site-packages\xgboost\training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


지니계수 : 0.28538973194158385

| 9         | 0.2854    | 0.9584    | 9.305     | 5.594     | 6.158     | 8.201     | 1.257     | 1.47      | 0.858     |


In [15]:
# 평가함수 점수가 최대일 떄 하이퍼 파라미터
max_params = optimizer.max['params']
max_params

{'colsample_bytree': 0.9583821245229369,
 'gamma': 9.305332775334449,
 'max_depth': 5.594282602920541,
 'min_child_weight': 6.158305055403563,
 'reg_alpha': 8.200928434091152,
 'reg_lambda': 1.2571039588093065,
 'scale_pos_weight': 1.4700266933495618,
 'subsample': 0.857971740304964}

In [16]:
# 정수형 하이퍼 파라미터 변환
max_params['max_depth'] = int(round(max_params['max_depth']))

# 값이 고정된 하이퍼 파라미터 추가
max_params.update(fixed_param)
max_params

{'colsample_bytree': 0.9583821245229369,
 'gamma': 9.305332775334449,
 'max_depth': 6,
 'min_child_weight': 6.158305055403563,
 'reg_alpha': 8.200928434091152,
 'reg_lambda': 1.2571039588093065,
 'scale_pos_weight': 1.4700266933495618,
 'subsample': 0.857971740304964,
 'objective': 'binary:logistic',
 'learning_rate': 0.02,
 'random_state': 1991}

In [19]:
# 모델 훈련 및 성능 검증 
from sklearn.model_selection import StratifiedKFold
# 층화 K 폴드 교차 검증기 생성
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)

# OOF 방식으로 훈련된 모델로 검증 데이터 타깃값을 예측한 확류를 담은 1차원 배열
oof_val_preds = np.zeros(X.shape[0])

# OOF 방식으로 훈련된 모델로 테스트 데이터 타깃값을 예측한 확률을 담은 1차원 배열
oof_test_preds = np.zeros(X_test.shape[0])


# OOF 방식으로 모델 훈련, 검증, 예측
for idx, (train_idx, valid_idx) in enumerate(folds.split(X,y)):
    # 각 폴드를 구분하는 문구 출력 
    print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#'*40)
    
    # 훈련용 데이터, 검증용 데이터 설정
    X_train, y_train = X[train_idx], y[train_idx]
    X_valid, y_valid = X[valid_idx], y[valid_idx]
    
    # XGBoost 전용 데이터셋 생성
    dtrain = xgb.DMatrix(X_train, y_train)
    dvalid = xgb.DMatrix(X_valid, y_valid)
    dtest  = xgb.DMatrix(X_test)
    
    # XGBoost 모델 훈련
    xgb_model = xgb.train(params=max_params,
                          dtrain=dtrain,
                          num_boost_round=2000,
                          evals=[(dvalid, 'valid')],
                          maximize=True,
                          feval=gini,
                          early_stopping_rounds=200,
                          verbose_eval=100)
    
    # 모델 성능이 가장 좋을 떄의 부스팅 반복 횟수 저장
    best_iter = xgb_model.best_iteration
    # 테스트 데이터를 활용해 OOF 예측
    oof_test_preds += xgb_model.predict(dtest,
                                        iteration_range=(0, best_iter))/folds.n_splits
    
    # 모델 성능 평가를 위한 검증 데이터 타깃값 예측 
    oof_val_preds[valid_idx] += xgb_model.predict(dvalid, iteration_range=(0, best_iter))
    
    # 검증 데이터 예측 확률에 대한 정규화 지니계수 
    gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])
    print(f'폴드 {idx+1} 지니계수 : {gini_score}\n')

######################################## 폴드 1 / 폴드 5 ########################################


c:\ProgramData\Anaconda3\lib\site-packages\xgboost\training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


[0]	valid-logloss:0.67671	valid-gini:0.16215
[100]	valid-logloss:0.19182	valid-gini:0.24941
[200]	valid-logloss:0.15837	valid-gini:0.27811
[300]	valid-logloss:0.15506	valid-gini:0.28780
[400]	valid-logloss:0.15452	valid-gini:0.29319
[500]	valid-logloss:0.15439	valid-gini:0.29533
[600]	valid-logloss:0.15433	valid-gini:0.29681
[700]	valid-logloss:0.15427	valid-gini:0.29816
[800]	valid-logloss:0.15425	valid-gini:0.29872
[900]	valid-logloss:0.15422	valid-gini:0.29916
[1000]	valid-logloss:0.15422	valid-gini:0.29949
[1100]	valid-logloss:0.15422	valid-gini:0.29951
[1200]	valid-logloss:0.15422	valid-gini:0.29941
[1300]	valid-logloss:0.15420	valid-gini:0.29947
[1338]	valid-logloss:0.15421	valid-gini:0.29945
폴드 1 지니계수 : 0.2996383794197772

######################################## 폴드 2 / 폴드 5 ########################################
[0]	valid-logloss:0.67670	valid-gini:0.15338
[100]	valid-logloss:0.19190	valid-gini:0.24056
[200]	valid-logloss:0.15862	valid-gini:0.26445
[300]	valid-logloss:0.15542

In [20]:
print('OOF 검증 데이터 지니계수 : ', eval_gini(y, oof_val_preds))

OOF 검증 데이터 지니계수 :  0.28905038063620453


In [21]:
submission['target'] = oof_test_preds
submission.to_csv('./output/submission_3.csv')

In [22]:
!kaggle competitions submit -c porto-seguro-safe-driver-prediction -f ./output/submission_3.csv -m submit_in_local

Successfully submitted to Porto Seguro’s Safe Driver Prediction


  0%|          | 0.00/24.2M [00:00<?, ?B/s]
  0%|          | 8.00k/24.2M [00:00<13:35, 31.1kB/s]
  0%|          | 120k/24.2M [00:00<00:59, 423kB/s]  
  2%|▏         | 480k/24.2M [00:00<00:16, 1.51MB/s]
  4%|▍         | 1.05M/24.2M [00:00<00:08, 2.95MB/s]
  6%|▌         | 1.39M/24.2M [00:02<00:40, 584kB/s] 
 10%|▉         | 2.38M/24.2M [00:02<00:17, 1.33MB/s]
 12%|█▏        | 2.86M/24.2M [00:02<00:13, 1.67MB/s]
 17%|█▋        | 4.03M/24.2M [00:02<00:07, 2.94MB/s]
 19%|█▉        | 4.70M/24.2M [00:02<00:06, 3.25MB/s]
 22%|██▏       | 5.28M/24.2M [00:02<00:05, 3.46MB/s]
 24%|██▍       | 5.81M/24.2M [00:02<00:05, 3.47MB/s]
 27%|██▋       | 6.45M/24.2M [00:02<00:04, 3.97MB/s]
 37%|███▋      | 8.85M/24.2M [00:03<00:01, 8.13MB/s]
 41%|████      | 9.85M/24.2M [00:03<00:02, 7.28MB/s]
 44%|████▍     | 10.7M/24.2M [00:03<00:01, 7.19MB/s]
 48%|████▊     | 11.5M/24.2M [00:03<00:01, 7.12MB/s]
 51%|█████     | 12.3M/24.2M [00:03<00:01, 6.81MB/s]
 54%|█████▎    | 13.0M/24.2M [00:03<00:01, 6.69MB/s]
 5